In [ ]:
import cv2
import socket
import struct
import pickle

# Initialize socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 9999))
server_socket.listen(1)

# Accept client connection
client_socket, addr = server_socket.accept()

# Initialize OpenCV Video Capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Compress frame
    frame = cv2.resize(frame, (640, 480))  # Resize for faster transmission
    encoded, buffer = cv2.imencode('.jpg', frame)
    
    # Serialize frame
    data = pickle.dumps(buffer)
    size = len(data)

    # Send frame size first, then the data
    client_socket.sendall(struct.pack(">L", size) + data)

cap.release()
client_socket.close()
server_socket.close()
